In [1]:
import mysql.connector
import sqlite3  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
mydb = mysql.connector.connect(
    user='root', 
    password='password',
    host='127.0.0.1', 
    database='Chinook',
    auth_plugin='mysql_native_password')

In [ ]:
album = pd.read_sql("SELECT * FROM Album", mydb)
artist = pd.read_sql("SELECT * FROM Artist", mydb)
customer = pd.read_sql("SELECT * FROM Customer", mydb)
employee = pd.read_sql("SELECT * FROM Employee", mydb)
genre = pd.read_sql("SELECT * FROM Genre", mydb)
invoice = pd.read_sql("SELECT * FROM Invoice", mydb)
invoice_line = pd.read_sql("SELECT * FROM InvoiceLine", mydb)
media_type = pd.read_sql("SELECT * FROM MediaType", mydb)
playlist = pd.read_sql("SELECT * FROM Playlist", mydb)
playlist_track = pd.read_sql("SELECT * FROM PlaylistTrack", mydb)
track = pd.read_sql("SELECT * FROM Track", mydb)

Normality test

In [ ]:
tables = {  
    'Album': pd.read_sql("SELECT * FROM Album", mydb),  
    'Artist': pd.read_sql("SELECT * FROM Artist", mydb),  
    'Customer': pd.read_sql("SELECT * FROM Customer", mydb),  
    'Employee': pd.read_sql("SELECT * FROM Employee", mydb),  
    'Genre': pd.read_sql("SELECT * FROM Genre", mydb),  
    'Invoice': pd.read_sql("SELECT * FROM Invoice", mydb),  
    'InvoiceLine': pd.read_sql("SELECT * FROM InvoiceLine", mydb),  
    'MediaType': pd.read_sql("SELECT * FROM MediaType", mydb),  
    'Playlist': pd.read_sql("SELECT * FROM Playlist", mydb),  
    'PlaylistTrack': pd.read_sql("SELECT * FROM PlaylistTrack", mydb),  
    'Track': pd.read_sql("SELECT * FROM Track", mydb),  
}

def check_normality(data):
    results = {}
    numeric_columns = data.select_dtypes(include=['number']).columns
    
    for column in numeric_columns:
        # 1. آزمون شاپیرو-ویلک
        shapiro_stat, shapiro_p = stats.shapiro(data[column].dropna())
        
        # 2. آزمون کولموگروف-اسمیرنف
        d_stat, ks_p = stats.kstest(data[column].dropna(), 'norm')  # مقایسه با توزیع نرمال
        
        # 3. رسم هیستوگرام و نمودار Q-Q
        plt.figure(figsize=(12, 6))
        
        # هیستوگرام
        plt.subplot(1, 2, 1)
        sns.histplot(data[column], kde=True)
        plt.title(f'Histogram of {column}')
        
        # نمودار Q-Q
        plt.subplot(1, 2, 2)
        stats.probplot(data[column].dropna(), dist="norm", plot=plt)
        plt.title(f'Q-Q Plot of {column}')
        
        plt.tight_layout()
        plt.show()
        
        results[column] = {
            "Shapiro-Wilk": shapiro_p,
            "Kolmogorov-Smirnov": ks_p,
            "Shapiro-Wilk Result": "Normal" if shapiro_p > 0.05 else "Not Normal",
            "Kolmogorov-Smirnov Result": "Normal" if ks_p > 0.05 else "Not Normal"
        }
    
    return results

for table_name, data in tables.items():
    print(f"\nNormality check for {table_name}:")
    normality_results = check_normality(data)
    for column, result in normality_results.items():
        print(f"\nColumn: {column}")
        print(f"Shapiro-Wilk p-value: {result['Shapiro-Wilk']}, Result: {result['Shapiro-Wilk Result']}")
        print(f"Kolmogorov-Smirnov p-value: {result['Kolmogorov-Smirnov']}, Result: {result['Kolmogorov-Smirnov Result']}")

Data cleaning

duplicated

In [ ]:
duplicates = album.duplicated().sum()
print("Number of duplicate rows:", duplicates)

missing data

In [ ]:
missing_values = track.isna().sum()
print("Missing Values:")
print(missing_values)

In [ ]:
track['Composer'].fillna('unknown', inplace=True)

In [ ]:
missing_values = invoice.isna().sum()
print("Missing Values:")
print(missing_values)

In [ ]:
invoice['BillingState'].fillna('unknown', inplace=True)
invoice['BillingPostalCode'].fillna('unknown', inplace=True)

In [ ]:
missing_values = customer.isna().sum()
print("Missing Values:")
print(missing_values)

In [ ]:
customer['Company'].fillna('unknown', inplace=True)
customer['State'].fillna('unknown', inplace=True)
customer['PostalCode'].fillna('unknown', inplace=True)
customer['Phone'].fillna('unknown', inplace=True)
customer['Fax'].fillna('unknown', inplace=True)

In [ ]:
missing_values = employee.isna().sum()
print("Missing Values:")
print(missing_values)

In [ ]:
employee['ReportsTo'].fillna('unknown', inplace=True)

outlier

In [ ]:
plt.boxplot(track['Milliseconds'], vert=False)
plt.ylabel('Name')
plt.xlabel('Milliseconds')
plt.show()

In [ ]:
plt.scatter(range(track.shape[0]), np.sort(track['Milliseconds'].values))
plt.xlabel('Milliseconds')
plt.ylabel('Name')
sns.despine()

In [ ]:
sns.distplot(track['Milliseconds'])
sns.despine()

In [ ]:
plt.boxplot(invoice['Total'], vert=False)
plt.ylabel('Invoiceid')
plt.xlabel('Total')
plt.show()

In [ ]:
plt.scatter(range(invoice.shape[0]), np.sort(invoice['Total'].values))
plt.xlabel('Total')
plt.ylabel('Invoiceid')
sns.despine()

In [ ]:
sns.distplot(invoice['Total'])
sns.despine()

In [ ]:
plt.boxplot(invoice_line['UnitPrice'], vert=False)
plt.ylabel('InvoiceLineId')
plt.xlabel('UnitPrice')
plt.show()

In [ ]:
plt.scatter(range(invoice_line.shape[0]), np.sort(invoice_line['UnitPrice'].values))
plt.xlabel('UnitPrice')
plt.ylabel('Invoiceid')
sns.despine()

In [ ]:
sns.distplot(invoice_line['UnitPrice'])
sns.despine()

3 ژانر محبوب رو پیدا کنید . تفاوت میانگین قیمت این دو توزیع رو بررسی کنید

In [ ]:
merged_data = pd.merge(invoice_line, track, on='TrackId')
merged_data = pd.merge(merged_data, genre, on='GenreId')

popular_genres = merged_data.groupby('Name_y')['Quantity'].sum().nlargest(3)

prices = merged_data.groupby('Name_y')['UnitPrice_y'].mean().loc[popular_genres.index]

t_stat, p_value = stats.ttest_ind(merged_data[merged_data['Name_y'] == prices.index[0]]['UnitPrice_y'],
                                   merged_data[merged_data['Name_y'] == prices.index[1]]['UnitPrice_y'])

print("3 ژانر محبوب:")
print(popular_genres)
print("\nمیانگین قیمت ژانرها:")
print(prices)
print("\nنتایج آزمون t-test:")
print(f"T-statistic: {t_stat}, P-value: {p_value}")

 وجود استقلال بین طول آهنگ و قیمت آن را بررسی کنید

In [ ]:
merged_data = pd.merge(invoice_line, track, on='TrackId')

correlation = stats.pearsonr(merged_data['Milliseconds'], merged_data['UnitPrice_x'])
print(f"Correlation between track length and price: {correlation[0]}, P-value: {correlation[1]}")


 استقالل ژانر آهنگ و نوع رسانه رو بررسی کنید 

In [ ]:
merged_data = pd.merge(track, media_type, on='MediaTypeId')

contingency_table = pd.crosstab(merged_data['Name_x'], merged_data['MediaTypeId'])
chi2_stat, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-squared statistic: {chi2_stat}, P-value: {p_value}")

محاسبه میانگین قیمت بر اساس نوع رسانه  

In [ ]:
merged_media_prices = track.merge(media_type, on='MediaTypeId')  

mean_price_per_media = merged_media_prices.groupby('MediaTypeId')['UnitPrice'].mean().reset_index(name='MeanPrice')  
mean_price_per_media = mean_price_per_media.merge(media_type[['MediaTypeId', 'Name']], on='MediaTypeId')  
 
print(mean_price_per_media[['Name', 'MeanPrice']])  

محاسبه مشتریانی که چند بار خرید کرده‌اند 

In [ ]:
customer_purchase_count = invoice.groupby('CustomerId')['InvoiceId'].count().reset_index(name='PurchaseCount')  
 
repeated_customers = customer_purchase_count[customer_purchase_count['PurchaseCount'] > 1]  
average_repeated_purchases = repeated_customers['PurchaseCount'].mean()  

print(f"Average number of purchases for repeat customers: {average_repeated_purchases}")  

محاسبه تعداد خریدها برای هر آلبوم 

In [ ]:
merged_album = invoice_line.merge(track, on='TrackId').merge(album, on='AlbumId')  
 
album_purchase_count = merged_album.groupby('AlbumId')['InvoiceLineId'].count().reset_index(name='PurchaseCount')  
popular_albums = album_purchase_count.merge(album[['AlbumId', 'Title']], on='AlbumId')  

popular_albums = popular_albums.sort_values(by='PurchaseCount', ascending=False)  

print(popular_albums[['Title', 'PurchaseCount']]) 

میانگین طول آهنگ ها در ژانرهای مختلف یکسانه؟ فاصله اطمینان 95 درصدی برای میانگین طول آهنگها در هر ژانر را محاسبه کنید

In [ ]:
def confidence_interval(data):
    if len(data) < 2:
        return (np.nan, np.nan)
    mean = np.mean(data)
    std_err = stats.sem(data)
    ci = stats.t.interval(0.95, len(data)-1, loc=mean, scale=std_err)
    return ci

sample_sizes = track.groupby('GenreId')['Milliseconds'].count()

valid_genres = sample_sizes[sample_sizes > 1].index

length_ci = track[track['GenreId'].isin(valid_genres)].groupby('GenreId')['Milliseconds'].apply(confidence_interval)

print("فاصله اطمینان 95 درصدی برای میانگین طول آهنگ‌ها در هر ژانر:")
print(length_ci)

میانگین فروش در کشورهای مختلف چقدره؟ فاصله اطمینان 95 درصدی برای میانگین فروش در هر کشور را محاسبه کنید

In [ ]:
merged_data = pd.merge(invoice, customer, on='CustomerId')

def confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    sem = stats.sem(data)  
    h = sem * stats.t.ppf((1 + confidence) / 2., len(data)-1)  
    return mean - h, mean + h


sales_ci = merged_data.groupby('Country')['Total'].apply(confidence_interval)
print("فاصله اطمینان 95 درصدی برای میانگین فروش در هر کشور:")
print(sales_ci)

• میانگین تعداد آهن گ های خریداری شده توسط هر کاربر چقدره؟ فاصله اط مینان 95 درصدی 
برای میانگین تعداد آهنگ های خریداری شده توسط هر کاربر را محاسبه کنید 

In [ ]:
merged_data = pd.merge(invoice_line, invoice, on='InvoiceId')
purchase_counts_ci = merged_data.groupby('CustomerId')['Quantity'].apply(confidence_interval)
print("فاصله اطمینان 95 درصدی برای میانگین تعداد آهنگ‌های خریداری شده توسط هر کاربر:")
print(purchase_counts_ci)